<a href="https://colab.research.google.com/github/edakiziltas08/GitDeneme/blob/master/prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
In this notebook, we explain all prerequisites for using our implementation.

Here are general steps you need to follow (this notebook covers the first two. train_all.ipynb covers the third):
1. Python. Make sure you have a python environment (conda or other) with libraries listed in requirements.txt
2. Data. The data preparation steps are provided in the DATA directory.
3. Commands. The rest of this notebook generates shell commands for starting the training process. We are running our code on a server with slurm for job management. You can change a little bit this notebook to adapt to your own environment. Providing GPU would make the running of these training jobs faster.

## Download XMarket Data

In [1]:
import os
import urllib.request as urlreq

tgt_markets = ['ca', 'de', 'fr', 'in', 'jp', 'mx', 'uk', 'us']
tgt_cat = 'Electronics'

##########
### Download XMarket data for papers categories and markets
#########
fix_url = 'https://ciir.cs.umass.edu/downloads/XMarket/FULL/'
orig_data_dl = 'DATA2/orig_data'
proc_data_out = 'DATA2/proc_data'
if not os.path.exists(orig_data_dl):
    os.makedirs(orig_data_dl)
if not os.path.exists(proc_data_out):
    os.makedirs(proc_data_out)

for tgt_market in tgt_markets:
    cur_url = f'{fix_url}{tgt_market}/{tgt_cat}/ratings_{tgt_market}_{tgt_cat}.txt.gz'
    urlreq.urlretrieve(cur_url, f'{orig_data_dl}/ratings_{tgt_market}_{tgt_cat}.txt.gz')
    print(f'Done: {cur_url}')

Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/ca/Electronics/ratings_ca_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/de/Electronics/ratings_de_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/fr/Electronics/ratings_fr_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/in/Electronics/ratings_in_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/jp/Electronics/ratings_jp_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/mx/Electronics/ratings_mx_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/uk/Electronics/ratings_uk_Electronics.txt.gz
Done: https://ciir.cs.umass.edu/downloads/XMarket/FULL/us/Electronics/ratings_us_Electronics.txt.gz


## Process and Filter

In [2]:
# set thr for K-core data cleaning. We use 5core, +1 valid and +1 test-> so at this point we filter 7core
user_thr = 7
item_thr = 7

# one can iterate a few times, we only perform one time filter
def get_kcore(ratings_all, user_thr, item_thr, repeat=1):
    for i in range(repeat):
        ratings_all.reset_index(drop=True, inplace=True)
        ratings_all = ratings_all.loc[ratings_all.groupby("itemId").filter(lambda x: len(x) >= item_thr).index]
        ratings_all.reset_index(drop=True, inplace=True)
        ratings_all = ratings_all.loc[ratings_all.groupby("userId").filter(lambda x: len(x) >= user_thr).index]
        ratings_all.reset_index(drop=True, inplace=True)
    return ratings_all

def rating_stats(ratings_all):
    n_rating = ratings_all.shape[0]
    n_user = len(set(ratings_all['userId'].unique()))
    n_item = len(set(ratings_all['itemId'].unique()))

    if (n_user*n_item)!=0:
        density = round((n_rating/(n_user*n_item) )*100, 5)
    else:
        density = 0

    return { '#users': n_user,
        '#items': n_item,
        '#rates': n_rating,
        'dens\%': density,
        }

In [3]:
##########
### load and clean us market as the M0
#########
import pandas as pd

tgt_market = 'us'
us_ratings_file = f'{orig_data_dl}/ratings_{tgt_market}_{tgt_cat}.txt.gz'

us_df = pd.read_csv(us_ratings_file, compression='gzip', header=None, sep=' ', names=["userId", "itemId", "rate", "date"] )
us_df_7core = get_kcore(us_df, user_thr=user_thr, item_thr=item_thr)

us_user_thr = 10
us_item_thr = 25
us_df_10core = get_kcore(us_df, user_thr=us_user_thr, item_thr=us_item_thr)

# write us data
us_df_7core.to_csv(f'{proc_data_out}/{tgt_market}_5core.txt', index=False, sep=' ')
us_df_10core.to_csv(f'{proc_data_out}/{tgt_market}_10core.txt', index=False, sep=' ')

print('US Market data stats:')
print(rating_stats(us_df))
print(rating_stats(us_df_7core))
print(rating_stats(us_df_10core))

##########
### load and clean target markets
#########
us_items_set = set(us_df_10core['itemId'].unique())

for tgt_market in tgt_markets:
    if tgt_market=='us':
        continue
    #read market ratings
    cur_ratings_file = f'{orig_data_dl}/ratings_{tgt_market}_{tgt_cat}.txt.gz'
    cur_df = pd.read_csv(cur_ratings_file, compression='gzip', header=None, sep=' ', names=["userId", "itemId", "rate", "date"] )

    # item exist for us
    cur_df = cur_df.loc[cur_df['itemId'].isin( us_items_set )]
    cur_df_7core = get_kcore(cur_df, user_thr=user_thr, item_thr=item_thr)
    cur_df_7core.to_csv(f'{proc_data_out}/{tgt_market}_5core.txt', index=False, sep=' ')
    print(f'\n-- {tgt_market} stats: ')
    print(rating_stats(cur_df_7core))

US Market data stats:
{'#users': 2784128, '#items': 35943, '#rates': 4169476, 'dens\\%': 0.00417}
{'#users': 35916, '#items': 31125, '#rates': 364339, 'dens\\%': 0.03259}
{'#users': 12490, '#items': 24528, '#rates': 178811, 'dens\\%': 0.05837}

-- ca stats: 
{'#users': 5675, '#items': 5772, '#rates': 55045, 'dens\\%': 0.16805}

-- de stats: 
{'#users': 2373, '#items': 2210, '#rates': 22247, 'dens\\%': 0.42421}

-- fr stats: 
{'#users': 2396, '#items': 1911, '#rates': 22905, 'dens\\%': 0.50025}

-- in stats: 
{'#users': 239, '#items': 470, '#rates': 2015, 'dens\\%': 1.79382}

-- jp stats: 
{'#users': 487, '#items': 955, '#rates': 4485, 'dens\\%': 0.96434}

-- mx stats: 
{'#users': 1878, '#items': 1645, '#rates': 17095, 'dens\\%': 0.55336}

-- uk stats: 
{'#users': 4847, '#items': 3302, '#rates': 44515, 'dens\\%': 0.27814}


In [7]:
import os
os.getcwd()
!ls

DATA2  sample_data


In [17]:

import matplotlib.pyplot as plt
import pickle



item_map = {}
user_map = {}
idi =0
idu =0
dic_data = {}

for tgt_market in tgt_markets:
  print(tgt_market)
  if tgt_market=='us':
    cur_ratings_file = f'DATA2/proc_data/{tgt_market}_10core.txt'
  else:
    cur_ratings_file = f'DATA2/proc_data/{tgt_market}_5core.txt'
  cur_df = pd.read_csv(cur_ratings_file, header=0, sep=' ', names=["userId", "itemId", "rate", "date"] )
  for row in cur_df.iterrows():
    user_id = row[1]['userId']
    item_id = row[1]['itemId']
    if user_id not in user_map:
      user_map[user_id] = idu
      idu += 1
    if item_id not in item_map:
      item_map[item_id] = idi
      idi += 1

for tgt_market in tgt_markets:
  if tgt_market=='us':
    cur_ratings_file = f'DATA2/proc_data/{tgt_market}_10core.txt'
  else:
    cur_ratings_file = f'DATA2/proc_data/{tgt_market}_5core.txt'
  dic_data[tgt_market] = pd.read_csv(cur_ratings_file, header=0, sep=' ', names=["userId", "itemId", "rate", "date"] )
  dic_data[tgt_market]['userId'] = dic_data[tgt_market]['userId'].map(user_map)
  dic_data[tgt_market]['itemId'] = dic_data[tgt_market]['itemId'].map(item_map)




with open('DATA2/dic_XMARKET.pkl', 'wb') as f:
    pickle.dump(dic_data, f)

with open('DATA2/item_map.pkl', 'wb') as f:
    pickle.dump(item_map, f)

with open('DATA2/user_map.pkl', 'wb') as f:
    pickle.dump(user_map, f)

concatenated_df = pd.concat(dic_data.values(), ignore_index=True)

concatenated_df.to_csv('DATA2/concatenated_df.txt', index=False, sep=' ')

print(concatenated_df.head())

print(rating_stats(concatenated_df))





ca
de
fr
in
jp
mx
uk
us
   userId  itemId  rate        date
0       0       0   5.0  2016-10-08
1       1       1   4.0  2015-12-27
2       2       2   5.0  2017-11-09
3       3       3   4.0  2018-08-30
4       4       4   3.0  2015-10-04
{'#users': 30362, '#items': 24528, '#rates': 347118, 'dens\\%': 0.04661}


In [ ]:
# most popular 100 items

